In [1]:
import csv
import math
import numpy as np
import pandas as pd
import re
import nltk

ModuleNotFoundError: No module named 'nltk'

In [2]:
def read_data_task1(filename, max=None, mode='orig', train=True, pad=True):
    """ method to read in data in csv format
        Arguments:
        filename        path to the file to read in
        max             maximum number of words per headline, needed for test set
    """
    texts = []
    labels = []
    ids = []
    indices = []
    tuples = []
    with open(filename, 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
        next(reader)
        regex = re.compile('[^a-zA-Z\d\s\']') # pattern to remove non-alphanumeric characters
        max_num_words = 0
        cnt = 0
        for line in reader:
            # if cnt > 100:
            #     break
            ids.append(line[0].strip())
            if train:
                labels.append(line[4].strip())
            text = line[1]
            start = text.find("<")
            end = text.find("/>")
            orig = text[start:end + 2]
            edit = line[2]
            if mode == 'edit':
                text = text.replace(orig, edit)
            else:
                while " " in orig:
                    white = orig.find(" ")
                    new_orig = orig[white+1:]
                    text = text.replace(orig, new_orig)
                    orig = new_orig
            text = text.strip()
            text = text.strip('\n')
            text = text.lower()
            text = re.sub(regex, "", text)
            orig = orig.strip()
            orig = orig.lower()
            orig = re.sub(regex, "", orig)
            edit = edit.strip()
            edit = edit.lower()
            tup = (orig, edit)
            tuples.append(tup)
            edit = re.sub(regex, "", edit)
            text_list = re.split('\s+', text)
            if mode == 'edit':
                edit_index = text_list.index(edit)
            else:
                edit_index = text_list.index(orig)
            if len(text_list) > max_num_words:
                max_num_words = len(text_list)
            indices.append(edit_index)
            texts.append(text)
            cnt = cnt + 1
    if pad:
        if max:
            max_num_words = max # ignore longest essay if max was passed as argument"
        padded = []
        for i in range(len(texts)):
            t = texts[i].split()
            t.insert(0, "<")
            if len(t) > max_num_words: # truncate longer essays
                t = t[0:max_num_words+1]
            while len(t) <= max_num_words: # pad shorter essays
                t.append(">")
            t = " ".join(t)
            padded.append(t)
        texts = padded
    labels = np.array(labels, dtype=float)
    if train:
        return texts, labels, indices, ids, tuples
    else:
        return texts, indices, ids, tuples

In [3]:
read_data_task1("semeval-2020-task-7-dataset/subtask-1/train.csv")

(['< france is hunting down its citizens who joined isis without trial in iraq > > > > > > > > > > > >',
  '< pentagon claims 2000 increase in russian trolls after syria strikes what does that mean > > > > > > > > > > >',
  '< iceland pm calls snap vote as pedophile furor crashes coalition > > > > > > > > > > > > > > >',
  '< in an apparent first iran and israel engage each other militarily > > > > > > > > > > > > > >',
  '< trump was told weeks ago that flynn misled vice president > > > > > > > > > > > > > > >',
  '< all 22 promises trump made in his speech to congress in one chart > > > > > > > > > > > >',
  '< new doj alert system will flag crimes against police > > > > > > > > > > > > > > > >',
  "< as someone who grew up among fundamentalist christians in the us i 'm surprised anyone 's surprised about roy moore > > > > >",
  "< canadians may pay more taxes than americans but here 's what they get for their money > > > > > > > > >",
  '< dutch minister resigns in drug baron row > 